In [1]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset
import cv2
import random
import pickle   
from tqdm import tqdm
import torch.nn as nn
import torch
import argparse
from dataset import iHand_dataset, RandomOrthogonalRotation
import time
from tapor_model_config import small_setting, base1_setting, base2_setting, large1_setting, large2_setting, base1_setting_varaint1, base1_setting_varaint2
from models import BlazeHandLandmark, ColorHandPose3D, Mano_MobileNetV2, Tapor, TaporTeacher
from torch.utils.tensorboard import SummaryWriter
from utils import draw_landmarks, draw_3D_landmarks, generate_heatmaps, calculate_mpjpe, calculate_pck, calculate_shift_error,BoneLengthLoss, KinematicChainLoss
import matplotlib.pyplot as plt



# Loading the original trainset

In [2]:
train_data_path = 'Trainset/'
train_file_names = [
    'P1_U1_L_1_D_0.pkl',
    'P1_U1_L_1_Z_0.pkl',
    'P1_U1_L_2_O_0.pkl',
    'P1_U1_L_2_S_0.pkl',
    'P1_U1_L_3_T_0.pkl',
    'P1_U1_L_4_O_0.pkl',
    'P1_U1_L_4_Z_0.pkl',
    'P1_U1_L_5_D_0.pkl',
    'P1_U1_L_5_T_0.pkl',
    'P1_U1_L_6_O_0.pkl',
    'P1_U1_L_7_S_0.pkl',
    'P1_U1_L_7_T_0.pkl',
    'P1_U1_L_8_D_0.pkl',
    'P1_U1_L_8_Z_0.pkl',
    'P1_U1_L_9_O_0.pkl',
    'P1_U1_L_9_S_0.pkl',
    'P1_U1_L_X_X_0.pkl',
    'P1_U1_L_X_X_1.pkl',
    'P1_U1_L_X_X_2.pkl',
    'P1_U1_L_X_X_3.pkl',
    'P1_U1_L_X_X_4.pkl',
    'P1_U1_R_0_D_0.pkl',
    'P1_U1_R_0_O_0.pkl',
    'P1_U1_R_0_Z_0.pkl',
    'P1_U1_R_1_D_1.pkl',
    'P1_U1_R_1_S_1.pkl',
    'P1_U1_R_1_T_1.pkl',
    'P1_U1_R_1_Z_1.pkl',
    'P1_U1_R_2_S_0.pkl',
    'P1_U1_R_2_S_1.pkl',
    'P1_U1_R_3_D_0.pkl',
    'P1_U1_R_3_T_0.pkl',
    'P1_U1_R_4_O_0.pkl',
    'P1_U1_R_4_Z_0.pkl',
    'P1_U1_R_5_S_0.pkl',
    'P1_U1_R_5_T_0.pkl',
    'P1_U1_R_6_D_0.pkl',
    'P1_U1_R_7_O_0.pkl',
    'P1_U1_R_7_S_0.pkl',
    'P1_U1_R_8_D_0.pkl',
    'P1_U1_R_8_T_0.pkl',
    'P1_U1_R_9_O_0.pkl',
    'P1_U1_R_9_Z_0.pkl',
    'P1_U1_R_X_X_0.pkl',
    'P1_U1_R_X_X_1.pkl',
    'P1_U1_R_X_X_2.pkl',
    'P1_U1_R_X_X_3.pkl',
    'P1_U1_R_X_X_4.pkl',
    'P1_U1_R_X_X_5.pkl',
    'P1_U2_L_0_O_0.pkl',
    'P1_U2_L_0_T_0.pkl',
    'P1_U2_L_0_Z_0.pkl',
    'P1_U2_L_0_Z_1.pkl',
    'P1_U2_L_1_X_0.pkl',
    'P1_U2_L_2_O_0.pkl',
    'P1_U2_L_3_T_0.pkl',
    'P1_U2_L_6_D_0.pkl',
    'P1_U2_L_9_T_0.pkl',
    'P1_U2_L_X_X_0.pkl',
    'P1_U2_L_X_X_1.pkl',
    'P1_U2_L_X_X_4.pkl',
    'P1_U2_R_0_D_0.pkl',
    'P1_U2_R_0_D_1.pkl',
    'P1_U2_R_0_O_0.pkl',
    'P1_U2_R_0_T_0.pkl',
    'P1_U2_R_0_T_2.pkl',
    'P1_U2_R_0_X_1.pkl',
    'P1_U2_R_0_Z_0.pkl',
    'P1_U2_R_1_D_1.pkl',
    'P1_U2_R_1_O_0.pkl',
    'P1_U2_R_1_T_0.pkl',
    'P1_U2_R_1_T_1.pkl',
    'P1_U2_R_1_X_0.pkl',
    'P1_U2_R_1_Z_0.pkl',
    'P1_U2_R_1_Z_1.pkl',
    'P1_U2_R_2_O_0.pkl',
    'P1_U2_R_2_X_0.pkl',
    'P1_U2_R_2_X_1.pkl',
    'P1_U2_R_3_T_0.pkl',
    'P1_U2_R_3_X_0.pkl',
    'P1_U2_R_3_X_1.pkl',
    'P1_U2_R_4_X_1.pkl',
    'P1_U2_R_4_Z_0.pkl',
    'P1_U2_R_4_Z_1.pkl',
    'P1_U2_R_5_S_0.pkl',
    'P1_U2_R_5_S_1.pkl',
    'P1_U2_R_5_X_0.pkl',
    'P1_U2_R_5_X_1.pkl',
    'P1_U2_R_6_D_0.pkl',
    'P1_U2_R_6_X_0.pkl',
    'P1_U2_R_6_X_1.pkl',
    'P1_U2_R_7_O_0.pkl',
    'P1_U2_R_7_X_1.pkl',
    'P1_U2_R_7_Z_0.pkl',
    'P1_U2_R_8_X_1.pkl',
    'P1_U2_R_8_Z_0.pkl',
    'P1_U2_R_8_Z_1.pkl',
    'P1_U2_R_9_T_0.pkl',
    'P1_U2_R_9_X_0.pkl',
    'P1_U2_R_X_X_3.pkl',
    'P1_U2_R_X_X_4.pkl',
    'P1_U2_R_X_X_5.pkl',
    'P1_U2_R_X_X_6.pkl',
    'P1_U3_L_X_X_0.pkl',
    'P1_U3_L_X_X_1.pkl',
    'P1_U3_L_X_X_2.pkl',
    'P1_U3_R_X_X_1.pkl',
    'P1_U3_R_X_X_2.pkl',
    'P1_U4_L_X_X_0.pkl',
    'P1_U4_R_X_X_0.pkl',
    'P1_U4_R_X_X_1.pkl',
    'P1_U4_R_X_X_2.pkl',
    'P1_U5_L_X_X_0.pkl',
    'P1_U5_L_X_X_1.pkl',
    'P1_U5_R_X_X_0.pkl',
    'P1_U5_R_X_X_1.pkl',
    'P1_U5_R_X_X_2.pkl',
    'P1_U6_L_X_X_0.pkl',
    'P1_U6_L_X_X_2.pkl',
    'P1_U6_R_X_X_0.pkl',
    'P1_U6_R_X_X_1.pkl',
    'P1_U6_R_X_X_2.pkl',
    'P1_U6_R_X_X_3.pkl',
    ]

fragment_size = 1
map_size = 0

In [3]:
dataset = iHand_dataset(train_data_path,
                        train_file_names,
                        fragment_size,
                        map_size = (map_size,map_size), 
                        filter_out_no_hand=True,
                        shuffle_fragment=True
                        )
print("The size of the training dataset is: ",len(dataset))

The size of the training dataset is:  33197


# preparing the Tapor teacher model

In [4]:
batch_size = 32
data_loader = torch.utils.data.DataLoader(dataset, batch_size, shuffle=False, drop_last=True)


config = base1_setting
device = torch.device('cuda:0')
spatial_encoder_param = config['spatial_encoder_param']
keypoints_encoder_param = config['keypoints_encoder_param']
cross_keypoints_fusion_param = config['cross_keypoints_fusion_param']
temporal_keypoints_fusion_param = config['temporal_keypoints_fusion_param']
handpose_encoder_param = config['handpose_encoder_param']
temporal_keypoints_fusion_param['num_history'] = fragment_size-1

tapor_model = TaporTeacher(spatial_encoder_param, 
                keypoints_encoder_param, 
                cross_keypoints_fusion_param, 
                temporal_keypoints_fusion_param,
                handpose_encoder_param,
                input_width=32, 
                input_height=24,
                batch_size = batch_size,
                train=True,
                device=device).to(device)

model_weight_path = "weights/" + "tapor.pth"
tapor_model.load_state_dict(torch.load(model_weight_path, map_location=device))
tapor_model.eval()

TaporTeacher(
  (spatial_encoder): MobileEncoder(
    (upsample): Upsample(scale_factor=10.0, mode='bilinear')
    (features): Sequential(
      (0): Sequential(
        (0): Conv2d(1, 1, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
          (3): Conv2d(1, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): B

# Inference: Extracting and Saving Tapor Model Features with Input Data and Ground Truth Labels

In [5]:
from tqdm import tqdm

thermal_maps = []
labels = []
cross_features = []
self_features = []

for i, data in enumerate(tqdm(data_loader)):
    thermal_map, ambient_temperature, l_depth_map, l_2d_joint,\
        l_2d_flag, l_3d_joint, l_3d_flag, l_hand_depth, l_left_right_flag,\
            heatmap_label = data
    l_3d_flag = l_3d_flag.float().to(device)  
    l_3d_joint = l_3d_joint.squeeze().float().to(device) 
    thermal_map = thermal_map.float().to(device)
    with torch.no_grad():
        pose, current_kp_feat, sp_feat, kp_feat, cross_attention_map, cross_attention_output, handpose_feat  = tapor_model(thermal_map)
    # permute the cross_attention_output to (batch_size, 21*48)
    cross_attention_output = cross_attention_output.permute(1,0,2).contiguous().view(-1, 21*48)
    handpose_feat = handpose_feat.contiguous().view(-1, 21*48)    # this is the feature after the self-attention
    
    thernal_map = thermal_map.cpu().numpy()
    thermal_maps.append(thernal_map)
    l_3d_joint = l_3d_joint.cpu().numpy()
    labels.append(l_3d_joint)
    cross_attention_output = cross_attention_output.cpu().numpy()
    cross_features.append(cross_attention_output)
    handpose_feat = handpose_feat.cpu().numpy()
    self_features.append(handpose_feat)

self_features = np.concatenate(self_features, axis=0)
cross_features = np.concatenate(cross_features, axis=0)
thermal_maps = np.concatenate(thermal_maps, axis=0)
labels = np.concatenate(labels, axis=0)


print("The shape of the self-attention features is: ", self_features.shape)
print("The shape of the cross-attention features is: ", cross_features.shape)
print("The shape of the thermal maps is: ", thermal_maps.shape)
print("The shape of the labels is: ", labels.shape)

    

100%|██████████| 1037/1037 [00:08<00:00, 119.83it/s]

The shape of the self-attention features is:  (33184, 1008)
The shape of the cross-attention features is:  (33184, 1008)
The shape of the thermal maps is:  (33184, 1, 24, 32)
The shape of the labels is:  (33184, 21, 3)


Save the data
commented out since it only needs to be run once

In [7]:
if not os.path.exists("NanoTapor_files"):
    os.makedirs("NanoTapor_files")
with open('NanoTapor_files/KD_training_data.pkl', 'wb') as f:
    pickle.dump([thermal_maps, labels, cross_features,self_features, ], f)
    